In [50]:
import pandas as pd
import glob
import csv
from collections import Counter
import os

הפונקציה הראשית נמצאת בסוף הדף, נית לראות בה את סדר זימון הפעולות עם הסבר.

In [ ]:
#--------סעיף 1

In [41]:
#המרת קובץ EXCEL ל CSV
def convert_excel_to_csv(input_file, output_file):
    df = pd.read_excel(input_file, engine='openpyxl')  
    df.to_csv(output_file, index=False, encoding='utf-8') 

#זימון הפונקציה
# convert_excel_to_csv("logs.txt.xlsx", "logs.csv")

In [42]:
#פונקציה המחלקת את הקובץ לחלקים קטנים יותר
#הפונקציה משתמשת ב- chunksize כדי לחסוך בזיכרון.
def split_csv_data(input_file, num_of_rows=100000, name_of_split_file="split_file_"):
    chunk_counter = 1

    for chunk in pd.read_csv(input_file, chunksize=num_of_rows, encoding='utf-8'):
        chunk.to_csv(f"{name_of_split_file}{chunk_counter}.csv", index=False, encoding="utf-8")
        chunk_counter += 1

#     print(f"הקובץ פוצל בהצלחה ל-{chunk_counter-1} חלקים!")


split_csv_data("logs.csv", 100000, "split")

# הכנסת כל המצביעים של הקבצים הקטנים לתוך מערך
split_arr = glob.glob("split*.csv") 
split_arr

In [43]:
#פונקציה המחשבת שכיחויות לכל חלק בנפרד.
def count_errors_in_chunk(chunk_file, file_name):
    error_counter = Counter()

    with open(chunk_file, 'r', encoding='utf-8') as file:
        for line in file:
            error_code = line.strip().split()[4]
            error_code = error_code.replace('"',"")
            error_counter[error_code] += 1

    output_file = f"{file_name}_with_freq.csv"

    with open(output_file, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(["Error_Code", "Count"])
        writer.writerows(error_counter.items())

#     print(f"שמירת התוצאה הסתיימה עבור {chunk_file} -> {output_file}")
#     print(f"common {error_counter.most_common(1)}")#כדי לראות את השגיאה השכיחה ביותר

#זימון הפעולה המחשבת שכיחויות עבור כל חלק
# for file in split_arr:
#     filename = f"{file}"
#     ext = '.csv'
#     name, ext = os.path.splitext(filename)
#     count_errors_in_chunk(file, name)

In [21]:
#הדפסה כדי לראות את כל התוכן של הקובץ עם השכיחויות עבור חלק מסויים
df=pd.read_csv("split10_with_freq.csv")
df

,Error_Code,Count
0,ERR_400,20036
1,ERR_500,20202
2,INFO_200,19782
3,WARN_101,19980
4,ERR_404,20000


In [22]:
#הכנסה למערך את כל המצביעים לקבצים המחולקים
errors_csv_arr = glob.glob("split*_with_freq.csv") 
errors_csv_arr

['split10_with_freq.csv',
 'split1_with_freq.csv',
 'split2_with_freq.csv',
 'split3_with_freq.csv',
 'split4_with_freq.csv',
 'split5_with_freq.csv',
 'split6_with_freq.csv',
 'split7_with_freq.csv',
 'split8_with_freq.csv',
 'split9_with_freq.csv']

In [25]:
errors = ['ERR_400', 'ERR_500', 'WARN_101', 'ERR_404']
new_df = pd.DataFrame(columns=errors)

new_df

,ERR_400,ERR_500,WARN_101,ERR_404


In [26]:
def build_file_of_error_with_freq(arr_of_error_files, dest_file):
    if dest_file.empty:
        dest_file = pd.DataFrame([[0] * len(dest_file.columns)], columns=dest_file.columns)
    for file_name in arr_of_error_files:
        try:
            with open(file_name, 'r', newline='', encoding='utf-8') as file:
                for line in file:
                    error_code = line.split(',')[0]
                    count = line.split(',')[1]
                    if error_code in dest_file.columns:
                        dest_file.loc[0,error_code] += int(count)
                    
        except FileNotFoundError:
            print(f"שגיאה: הקובץ {file_name} לא נמצא.")
        except OSError as e:
            print(f"שגיאת מערכת בקובץ {file_name}: {e}")
    return dest_file


# updated_df = build_file_of_error_with_freq(errors_csv_arr, new_df)

# updated_df.to_csv("all_errors_with_frequency.csv", index=False, encoding='utf-8')
# df= pd.read_csv("all_errors_with_frequency.csv")
# df

,ERR_400,ERR_500,WARN_101,ERR_404
0,200078,199808,200098,200094


In [33]:
all_error_counter = Counter()
errors = df.iloc[0].to_dict()
errors
all_error_counter = Counter(errors)
all_error_counter.most_common(2)
# for error in updated_df.columns():
#     cnt[error] += updated_df.loc[0, error]

[('WARN_101', 200098), ('ERR_404', 200094)]

In [46]:
#פונקציה ראשית הנעזרת בכל הפונקציות שלעיל
#מקבלת כקלט קובץ טקסט גדול ומחזירה רשימה של N קודי השגיאה השכיחים ביותר

def N_freq_error(file, N):
    
    if(N > 4):
        print("N need to be between 0-4")
        return
    
    else:
        #זימון הפונ' הממירה את הקובץ הגדול ל CSV
        convert_excel_to_csv(file, "logs.csv")

        #זימון הפונ' המחלקת את הקובץ הגדול
        split_csv_data("logs.csv", 100000, "split")

        #הכנסת כל המצביעים של הקבצים הקטנים לתוך מערך
        split_arr = glob.glob("split*.csv") 

        #זימון הפעולה המחשבת שכיחויות עבור כל חלק
        for file in split_arr:
            filename = f"{file}"
            ext = '.csv'
            name, ext = os.path.splitext(filename)
            count_errors_in_chunk(file, name)

        #הכנסה למערך את כל המצביעים לקבצים המחולקים עם השכיחויות
        errors_csv_arr = glob.glob("split*_with_freq.csv") 

        errors = ['ERR_400', 'ERR_500', 'WARN_101', 'ERR_404']
        new_df = pd.DataFrame(columns=errors)

        updated_df = build_file_of_error_with_freq(errors_csv_arr, new_df)

        updated_df.to_csv("all_errors_with_frequency.csv", index=False, encoding='utf-8')
        df= pd.read_csv("all_errors_with_frequency.csv")


        all_error_counter = Counter()
        errors = df.iloc[0].to_dict()
        all_error_counter = Counter(errors)
        return all_error_counter.most_common(N)

N = 3
print(f"The {N} most common errors: ", N_freq_error("logs.txt.xlsx", N))    

The 3 most common errors:  [('WARN_101', 200098), ('ERR_404', 200094), ('ERR_400', 200078)]


ניתוח סיבוכיות זמן ריצה וסיבוכיות זיכרון:

זמן ריצה-

כמות השורות בקובץ שהתקבל כפלט - n

כמות הקבצים שנוצרו מחילוק הקובץ הגדול - m (n/100000)

חישוב הסיבוכיות:

המרת הקובץ לCSV O(n) - עוברים על כל שורה.

חלוקת הקובץ O(n) - עוברים על כל שורה.

ניתוח קוד השגיאה O(n) - עוברים על כל החלקים ובכל אחד עוברים על כל השורות, סה"כ n שורות.

מיזוג כל הקבצים O(m) - כמות השגיאות היה קבועה ולכן לא נחשבת.

בניית Counter O(1) - Counter עובד על Hash Table.

סה"כ סיבוכיות זיכרון - O(n)

זיכרון:

הזיכרון קבוע O(1)

בגלל שהפונקציות משתמשות ב chunksize

אף פעם לא כל השורות יטענו לזיכרון. המקסימום שיהיה בזיכרון הוא 100000 שורות.